In [4]:
%pip install pyarrow


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [5]:
import ibis
from ibis import _
import ibis.selectors as s
con = ibis.duckdb.connect(extensions=["spatial"])

import pandas as pd
import numpy as np
import seaborn.objects as so

from pystac_client import Client
import odc.stac
import rioxarray
from exactextract import exact_extract
import leafmap.maplibregl as leafmap
from IPython.display import IFrame


In [6]:
holc = (
    con.read_geo(
        "/vsicurl/https://dsl.richmond.edu/panorama/redlining/static/mappinginequality.gpkg"
    )
    .filter(_.city == "Fresno", _.residential)
    .select("area_id", "city", "state", "grade", "label", "fill", "geom")
)

fresno = holc.execute().set_crs("EPSG:4326")
box = fresno.total_bounds
box


array([-119.82532,   36.71406, -119.72765,   36.78665])

In [7]:
items_lst = (
    Client.open("https://earth-search.aws.element84.com/v1")
    .search(
        collections=["landsat-c2l2-st"],
        bbox=box,
        datetime="2024-06-01/2024-09-01",
        query={"eo:cloud_cover": {"lt": 10}}
    )
    .item_collection()
)
len(items_lst)

0